<a href="https://colab.research.google.com/github/Montrealboyz/EQ_Blast_discriminate/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision --upgrade
!pip install timm

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import os
import gc
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision import transforms, datasets
from torchvision.utils import save_image
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from google.colab import drive
import torch.optim as optim
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
  print(f"Nvidia Cuda/GPU is available!")
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)
# Commented out IPython magic to ensure Python compatibility.


drive.mount('/content/gdrive' )
os.chdir('/content/gdrive/MyDrive/AGU_discrimination') # change path if needed

Nvidia Cuda/GPU is available!
Fri Nov 29 00:07:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              13W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-------------------------------------

In [ ]:
# Transformations for the training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images by up to 10 degrees
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])

# Transformations for the validation and testing data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize images
])


file_path = './blast_tensor_SNR18.pt'
blast_tensor = torch.load(file_path)
file_path = './eq_tensor_SNR18.pt'
eq_tensor = torch.load(file_path)

<ipython-input-3-69d4466866ca>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  blast_tensor = torch.load(file_path)
<ipython-input-3-69d4466866ca>:19: FutureWarning: You a

In [ ]:
num_samples = eq_tensor.size(0)
eq_labels = np.ones((num_samples, 1))
num_samples =blast_tensor.size(0)
blast_labels = np.zeros((num_samples, 1))
blast_hour = np.load('blast_tensor_SNR18_hour.npy')
eq_hour = np.load('eq_tensor_SNR18_hour.npy')
eq_labels = torch.from_numpy(eq_labels).float()
blast_labels = torch.from_numpy(blast_labels).float()
eq_tensor = eq_tensor.float()
blast_tensor = blast_tensor.float()
data = torch.cat((eq_tensor, blast_tensor), dim=0)
labels = torch.cat((eq_labels, blast_labels), dim=0)
# Combine the hour data similarly to how you combined the images and labels
hours = np.concatenate((eq_hour, blast_hour), axis=0)
hours = torch.from_numpy(hours).float()  # Convert to a PyTorch tensor
hours = hours.unsqueeze(1)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, hours, transform=None):
        self.images = images
        self.labels = labels
        self.hours = hours
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        hour = self.hours[idx]  # Retrieve the corresponding hour

        if self.transform:
            image = self.transform(image)

        return image, label, hour  # Return the image, its corresponding hour, and label

In [ ]:
def shuffle_split_data(data, labels, hours, train_frac=0.7, val_frac=0.15):
    # Shuffle the data, labels, and hours together
    perm = torch.randperm(data.size(0))
    data_shuffled = data[perm]
    labels_shuffled = labels[perm]
    hours_shuffled = hours[perm]  # Shuffle hours with the same permutation

    # Split the data
    train_size = int(data.size(0) * train_frac)
    val_size = int(data.size(0) * val_frac)

    train_data = data_shuffled[:train_size]
    train_labels = labels_shuffled[:train_size]
    train_hours = hours_shuffled[:train_size]  # Split hours for training

    val_data = data_shuffled[train_size:train_size + val_size]
    val_labels = labels_shuffled[train_size:train_size + val_size]
    val_hours = hours_shuffled[train_size:train_size + val_size]  # Split hours for validation

    test_data = data_shuffled[train_size + val_size:]
    test_labels = labels_shuffled[train_size + val_size:]
    test_hours = hours_shuffled[train_size + val_size:]  # Split hours for testing

    return train_data, train_labels, train_hours, val_data, val_labels, val_hours, test_data, test_labels, test_hours

In [ ]:
# Modify the function call to include `hours`
train_data, train_labels, train_hours, val_data, val_labels, val_hours, test_data, test_labels,test_hours = shuffle_split_data(data, labels, hours)  # Add 'hours' splitting here

In [ ]:
batch_size = 64  # Adjust as per your requirement

# Assuming you have train_hours, val_hours, and test_hours defined from your data splitting process
train_dataset = CustomDataset(train_data, train_labels, train_hours, transform=train_transforms)
val_dataset = CustomDataset(val_data, val_labels, val_hours, transform=test_transforms)
test_dataset = CustomDataset(test_data, test_labels, test_hours, transform=test_transforms)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## **Resnet-50**

In [ ]:
class ResNet50WithHours(nn.Module):
    def __init__(self, num_hour_features=1, dropout_prob=0):
        super(ResNet50WithHours, self).__init__()
        # Load a pre-trained ResNet-50 model
        self.resnet50 = models.resnet50(pretrained=True)
        # Remove the final fully connected layer
        self.resnet50 = nn.Sequential(*list(self.resnet50.children())[:-1])

        # Update for ResNet-50
        self.num_ftrs = 2048  # Output features from ResNet-50

        # Additional layer for hour feature(s)
        self.hour_fc = nn.Linear(num_hour_features, 24)  # Transform the hour feature

        # Dropout layers
        self.dropout_resnet = nn.Dropout(p=dropout_prob)
        self.dropout_hour = nn.Dropout(p=dropout_prob)
        self.dropout_combined = nn.Dropout(p=dropout_prob)

        # Final layer for binary classification
        self.final_fc = nn.Linear(self.num_ftrs + 24, 1)

    def forward(self, images, hours):
        # Forward pass through ResNet-50
        resnet_features = self.resnet50(images)
        resnet_features = resnet_features.view(-1, self.num_ftrs)  # Flatten the features
        resnet_features = self.dropout_resnet(resnet_features)  # Apply dropout

        # Forward pass through the hour feature layer
        hour_features = torch.relu(self.hour_fc(hours))
        hour_features = self.dropout_hour(hour_features)  # Apply dropout

        # Combine the features
        combined_features = torch.cat((resnet_features, hour_features), dim=1)
        combined_features = self.dropout_combined(combined_features)  # Apply dropout

        # Final classification
        output = self.final_fc(combined_features)
        return output

learning rate 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50WithHours(num_hour_features=1, dropout_prob=0).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 210MB/s]


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.24730832874774933, Accuracy: 88.56037816931672%
Iteration: 1000, Loss: 0.1337786465883255, Accuracy: 91.72324881822088%
Iteration: 1500, Loss: 0.25920796394348145, Accuracy: 92.7632144391921%
Iteration: 2000, Loss: 0.19757899641990662, Accuracy: 93.02965191233348%
Iteration: 2500, Loss: 0.21460166573524475, Accuracy: 91.4568113450795%
Iteration: 3000, Loss: 0.2411913424730301, Accuracy: 92.97808336914483%
Iteration: 3500, Loss: 0.14584645628929138, Accuracy: 92.51396648044692%
Iteration: 4000, Loss: 0.14501284062862396, Accuracy: 94.23291792006876%
Iteration: 4500, Loss: 0.1493687778711319, Accuracy: 93.60550064460679%
Iteration: 5000, Loss: 0.13278797268867493, Accuracy: 91.67168027503223%
Iteration: 5500, Loss: 0.08884447813034058, Accuracy: 94.3704340352385%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1546
Accuracy: 0.9421
Precision: 0.9384
Recall: 0.9579
F1 Score: 0.9480
ROC AUC Score: 0.9817
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 472848896 bytes
Memory Reserved after model1 deletion: 899678208 bytes


learning rate 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50WithHours(num_hour_features=1, dropout_prob=0).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.24691146612167358, Accuracy: 93.21014181349376%
Iteration: 1000, Loss: 0.11035985499620438, Accuracy: 93.67425870219166%
Iteration: 1500, Loss: 0.197535440325737, Accuracy: 94.15556510528577%
Iteration: 2000, Loss: 0.15805745124816895, Accuracy: 94.91190373871937%
Iteration: 2500, Loss: 0.2803337275981903, Accuracy: 94.42200257842715%
Iteration: 3000, Loss: 0.07939139753580093, Accuracy: 94.92049849591749%
Iteration: 3500, Loss: 0.05122937262058258, Accuracy: 94.81736140954018%
Iteration: 4000, Loss: 0.12070658802986145, Accuracy: 95.22131499785131%
Iteration: 4500, Loss: 0.16030585765838623, Accuracy: 95.25569402664375%
Iteration: 5000, Loss: 0.22791031002998352, Accuracy: 95.60807907176623%
Iteration: 5500, Loss: 0.16496744751930237, Accuracy: 95.53072625698324%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1459
Accuracy: 0.9366
Precision: 0.9754
Recall: 0.9078
F1 Score: 0.9403
ROC AUC Score: 0.9857
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

learning rate 0.0001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50WithHours(num_hour_features=1, dropout_prob=0).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 217MB/s]


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.14199517667293549, Accuracy: 94.05242801890847%
Iteration: 1000, Loss: 0.16648559272289276, Accuracy: 95.00644606789858%
Iteration: 1500, Loss: 0.14333130419254303, Accuracy: 95.68543188654921%
Iteration: 2000, Loss: 0.16717742383480072, Accuracy: 95.79716373012462%
Iteration: 2500, Loss: 0.16665399074554443, Accuracy: 95.68543188654921%
Iteration: 3000, Loss: 0.0505056194961071, Accuracy: 95.99484314568113%
Iteration: 3500, Loss: 0.19950005412101746, Accuracy: 96.08079071766223%
Iteration: 4000, Loss: 0.12164194881916046, Accuracy: 95.26428878384186%
Iteration: 4500, Loss: 0.14899513125419617, Accuracy: 95.89170605930383%
Iteration: 5000, Loss: 0.056694015860557556, Accuracy: 96.21830683283197%
Iteration: 5500, Loss: 0.09944994747638702, Accuracy: 96.38160721959605%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1021
Accuracy: 0.9631
Precision: 0.9696
Recall: 0.9634
F1 Score: 0.9665
ROC AUC Score: 0.9914
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 473935872 bytes
Memory Reserved after model1 deletion: 803209216 bytes


learning rate 0.00001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50WithHours(num_hour_features=1, dropout_prob=0).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.12193170189857483, Accuracy: 91.24194241512677%
Iteration: 1000, Loss: 0.09444105625152588, Accuracy: 92.60850880962613%
Iteration: 1500, Loss: 0.06555213034152985, Accuracy: 93.82896433175763%
Iteration: 2000, Loss: 0.17121830582618713, Accuracy: 93.69144821658789%
Iteration: 2500, Loss: 0.0651993528008461, Accuracy: 94.32746024924796%
Iteration: 3000, Loss: 0.057550884783267975, Accuracy: 94.23291792006876%
Iteration: 3500, Loss: 0.0912308469414711, Accuracy: 94.86033519553072%
Iteration: 4000, Loss: 0.026316873729228973, Accuracy: 94.8259561667383%
Iteration: 4500, Loss: 0.10558198392391205, Accuracy: 94.80876665234207%
Iteration: 5000, Loss: 0.1299992799758911, Accuracy: 94.8259561667383%
Iteration: 5500, Loss: 0.016605194658041, Accuracy: 94.96347228190804%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1139
Accuracy: 0.9580
Precision: 0.9660
Recall: 0.9575
F1 Score: 0.9617
ROC AUC Score: 0.9888
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 438398976 bytes
Memory Reserved after model1 deletion: 692060160 bytes


## **Resnet-18**

In [ ]:
class ResNetWithHours(nn.Module):
    def __init__(self, num_hour_features=1):
        super(ResNetWithHours, self).__init__()
        # Load a pre-trained ResNet-18 model
        self.resnet18 = models.resnet18(pretrained=True)
        # Remove the final fully connected layer
        self.resnet18 = nn.Sequential(*list(self.resnet18.children())[:-1])

        self.num_ftrs = 512  # Updated for ResNet-18

        # Additional layer for hour feature(s)
        self.hour_fc = nn.Linear(num_hour_features, 16)  # Example transformation of the hour feature

        # Final layer for binary classification
        self.final_fc = nn.Linear(self.num_ftrs + 16, 1)  # Use self.num_ftrs here

    def forward(self, images, hours):
        # Forward pass through ResNet-18
        resnet_features = self.resnet18(images)
        resnet_features = resnet_features.view(-1, self.num_ftrs)  # Flatten the features

        # Forward pass through the hour feature layer
        hour_features = torch.relu(self.hour_fc(hours))

        # Combine the features
        combined_features = torch.cat((resnet_features, hour_features), dim=1)

        # Final classification
        output = self.final_fc(combined_features)
        return output

learning rate 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 224MB/s]


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.22581788897514343, Accuracy: 85.87021916630856%
Iteration: 1000, Loss: 0.2912416160106659, Accuracy: 88.6291362269016%
Iteration: 1500, Loss: 0.1125142052769661, Accuracy: 91.42243231628707%
Iteration: 2000, Loss: 0.14540427923202515, Accuracy: 92.78899871078642%
Iteration: 2500, Loss: 0.13263902068138123, Accuracy: 92.22174473571121%
Iteration: 3000, Loss: 0.18058906495571136, Accuracy: 93.63987967339922%
Iteration: 3500, Loss: 0.200923353433609, Accuracy: 94.0008594757198%
Iteration: 4000, Loss: 0.09421859681606293, Accuracy: 94.02664374731414%
Iteration: 4500, Loss: 0.13032330572605133, Accuracy: 92.7632144391921%
Iteration: 5000, Loss: 0.12590914964675903, Accuracy: 94.98925655350236%
Iteration: 5500, Loss: 0.1229606345295906, Accuracy: 94.68844005156855%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1587
Accuracy: 0.9394
Precision: 0.9214
Recall: 0.9742
F1 Score: 0.9471
ROC AUC Score: 0.9850
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 227232256 bytes
Memory Reserved after model1 deletion: 371195904 bytes


learning rate 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.05217624455690384, Accuracy: 93.45938977223894%
Iteration: 1000, Loss: 0.24652116000652313, Accuracy: 93.59690588740868%
Iteration: 1500, Loss: 0.1710663139820099, Accuracy: 94.91190373871937%
Iteration: 2000, Loss: 0.09191072732210159, Accuracy: 94.97206703910615%
Iteration: 2500, Loss: 0.1600845605134964, Accuracy: 95.22990975504942%
Iteration: 3000, Loss: 0.10231349617242813, Accuracy: 95.05801461108723%
Iteration: 3500, Loss: 0.08674722909927368, Accuracy: 95.13536742587021%
Iteration: 4000, Loss: 0.1464272439479828, Accuracy: 95.3932101418135%
Iteration: 4500, Loss: 0.12844708561897278, Accuracy: 95.37602062741728%
Iteration: 5000, Loss: 0.1259939968585968, Accuracy: 95.18693596905888%
Iteration: 5500, Loss: 0.11111168563365936, Accuracy: 95.8745165449076%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1121
Accuracy: 0.9562
Precision: 0.9709
Recall: 0.9498
F1 Score: 0.9602
ROC AUC Score: 0.9895
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 266169856 bytes
Memory Reserved after model1 deletion: 417333248 bytes


learning rate 0.0001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.097661592066288, Accuracy: 92.25612376450366%
Iteration: 1000, Loss: 0.1474466621875763, Accuracy: 95.25569402664375%
Iteration: 1500, Loss: 0.07751607149839401, Accuracy: 95.46196819939837%
Iteration: 2000, Loss: 0.040054336190223694, Accuracy: 95.57370004297378%
Iteration: 2500, Loss: 0.04979481175541878, Accuracy: 95.81435324452085%
Iteration: 3000, Loss: 0.12764373421669006, Accuracy: 96.07219596046411%
Iteration: 3500, Loss: 0.1401008665561676, Accuracy: 96.063601203266%
Iteration: 4000, Loss: 0.22898411750793457, Accuracy: 96.27847013321873%
Iteration: 4500, Loss: 0.15430347621440887, Accuracy: 95.93467984529437%
Iteration: 5000, Loss: 0.07680060714483261, Accuracy: 95.68543188654921%
Iteration: 5500, Loss: 0.028390103951096535, Accuracy: 96.12376450365277%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1359
Accuracy: 0.9427
Precision: 0.9853
Recall: 0.9105
F1 Score: 0.9464
ROC AUC Score: 0.9901
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 227828224 bytes
Memory Reserved after model1 deletion: 463470592 bytes


learning rate 0.00001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.15535256266593933, Accuracy: 89.57455951869359%
Iteration: 1000, Loss: 0.22010602056980133, Accuracy: 91.41383755908896%
Iteration: 1500, Loss: 0.2211778163909912, Accuracy: 92.40223463687151%
Iteration: 2000, Loss: 0.20367984473705292, Accuracy: 93.08981521272024%
Iteration: 2500, Loss: 0.18046337366104126, Accuracy: 93.53674258702192%
Iteration: 3000, Loss: 0.09347458183765411, Accuracy: 93.55393210141814%
Iteration: 3500, Loss: 0.09028075635433197, Accuracy: 94.06961753330468%
Iteration: 4000, Loss: 0.10890308767557144, Accuracy: 94.03523850451225%
Iteration: 4500, Loss: 0.08687527477741241, Accuracy: 94.36183927804039%
Iteration: 5000, Loss: 0.07994537800550461, Accuracy: 94.7400085947572%
Iteration: 5500, Loss: 0.06215696781873703, Accuracy: 94.94628276751182%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1222
Accuracy: 0.9562
Precision: 0.9553
Recall: 0.9666
F1 Score: 0.9609
ROC AUC Score: 0.9867
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 227558912 bytes
Memory Reserved after model1 deletion: 497025024 bytes


## **VGG-16**

In [ ]:
class VGG16WithHours(nn.Module):
    def __init__(self, num_hour_features=1):
        super(VGG16WithHours, self).__init__()
        # Load a pre-trained VGG16 model
        self.vgg16 = models.vgg16(pretrained=True)

        # Modify the classifier
        # Remove the last classification layer (the last Linear layer)
        self.vgg16.classifier = nn.Sequential(
            *list(self.vgg16.classifier.children())[:-1]  # Exclude the last layer
        )

        # Find the number of features output by the last Linear layer
        num_ftrs = None
        for layer in reversed(self.vgg16.classifier):
            if isinstance(layer, nn.Linear):
                num_ftrs = layer.out_features
                break
        if num_ftrs is None:
            raise ValueError("No Linear layer found in vgg16.classifier")

        # Additional layer for hour feature(s)
        self.hour_fc = nn.Linear(num_hour_features, 16)  # Transform the hour feature

        # Final layer for binary classification
        self.final_fc = nn.Linear(num_ftrs + 16, 1)

    def forward(self, images, hours):
        # Forward pass through VGG16
        features = self.vgg16(images)  # Output size: [batch_size, num_ftrs]

        # Forward pass through the hour feature layer
        hour_features = torch.relu(self.hour_fc(hours))  # Output size: [batch_size, 16]

        # Combine the features
        combined_features = torch.cat((features, hour_features), dim=1)  # [batch_size, num_ftrs + 16]

        # Final classification
        output = self.final_fc(combined_features)  # Output size: [batch_size, 1]
        return output


learning rate 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the appropriate device
model = VGG16WithHours(num_hour_features=1).to(device)

negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.4277605414390564, Accuracy: 77.06059303824667%
Iteration: 1000, Loss: 0.39756089448928833, Accuracy: 78.03180060163301%
Iteration: 1500, Loss: 0.3726741671562195, Accuracy: 76.9058874086807%
Iteration: 2000, Loss: 0.44388020038604736, Accuracy: 76.95745595186936%
Iteration: 2500, Loss: 0.40823936462402344, Accuracy: 78.03180060163301%
Iteration: 3000, Loss: 0.5207967758178711, Accuracy: 76.95745595186936%
Iteration: 3500, Loss: 0.48589831590652466, Accuracy: 76.95745595186936%
Iteration: 4000, Loss: 0.46096891164779663, Accuracy: 80.48990116029222%
Iteration: 4500, Loss: 0.3895186185836792, Accuracy: 80.6532015470563%
Iteration: 5000, Loss: 0.42466872930526733, Accuracy: 79.51869359690589%
Iteration: 5500, Loss: 0.3910786807537079, Accuracy: 81.74473571121615%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.3978
Accuracy: 0.8030
Precision: 0.8648
Recall: 0.7589
F1 Score: 0.8084
ROC AUC Score: 0.8651
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 2197217792 bytes
Memory Reserved after model1 deletion: 3093299200 bytes


learning rate 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the appropriate device
model = VGG16WithHours(num_hour_features=1).to(device)

negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.42564404010772705, Accuracy: 81.35797163730125%
Iteration: 1000, Loss: 0.11262533068656921, Accuracy: 84.1340782122905%
Iteration: 1500, Loss: 0.21463292837142944, Accuracy: 93.18435754189944%
Iteration: 2000, Loss: 0.15441660583019257, Accuracy: 92.38504512247529%
Iteration: 2500, Loss: 0.15930379927158356, Accuracy: 93.78599054576708%
Iteration: 3000, Loss: 0.3134002089500427, Accuracy: 93.10700472711646%
Iteration: 3500, Loss: 0.18326322734355927, Accuracy: 93.55393210141814%
Iteration: 4000, Loss: 0.09494480490684509, Accuracy: 94.74860335195531%
Iteration: 4500, Loss: 0.22549748420715332, Accuracy: 93.19295229909756%
Iteration: 5000, Loss: 0.14156734943389893, Accuracy: 94.80876665234207%
Iteration: 5500, Loss: 0.15190403163433075, Accuracy: 93.5195530726257%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1578
Accuracy: 0.9451
Precision: 0.9336
Recall: 0.9686
F1 Score: 0.9508
ROC AUC Score: 0.9855
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 2196649472 bytes
Memory Reserved after model1 deletion: 2980052992 bytes


learning rate 0.0001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the appropriate device
model = VGG16WithHours(num_hour_features=1).to(device)

negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.13139332830905914, Accuracy: 94.30167597765363%
Iteration: 1000, Loss: 0.1117510199546814, Accuracy: 94.5509239363988%
Iteration: 1500, Loss: 0.06414756178855896, Accuracy: 95.45337344220026%
Iteration: 2000, Loss: 0.0693783387541771, Accuracy: 94.94628276751182%
Iteration: 2500, Loss: 0.11553597450256348, Accuracy: 95.01504082509669%
Iteration: 3000, Loss: 0.08226904273033142, Accuracy: 95.97765363128492%
Iteration: 3500, Loss: 0.034720323979854584, Accuracy: 95.23850451224753%
Iteration: 4000, Loss: 0.0774001032114029, Accuracy: 96.17533304684143%
Iteration: 4500, Loss: 0.06056244671344757, Accuracy: 96.05500644606789%
Iteration: 5000, Loss: 0.05751349404454231, Accuracy: 96.2440911044263%
Iteration: 5500, Loss: 0.0410083532333374, Accuracy: 96.02062741727546%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1219
Accuracy: 0.9498
Precision: 0.9830
Recall: 0.9243
F1 Score: 0.9528
ROC AUC Score: 0.9904
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 2196663808 bytes
Memory Reserved after model1 deletion: 2980052992 bytes


learning rate 0.00001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the appropriate device
model = VGG16WithHours(num_hour_features=1).to(device)

negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.2810930609703064, Accuracy: 93.22733132788998%
Iteration: 1000, Loss: 0.19488099217414856, Accuracy: 94.06961753330468%
Iteration: 1500, Loss: 0.16232308745384216, Accuracy: 94.68844005156855%
Iteration: 2000, Loss: 0.15970219671726227, Accuracy: 94.88611946712506%
Iteration: 2500, Loss: 0.14172379672527313, Accuracy: 95.19553072625699%
Iteration: 3000, Loss: 0.079299196600914, Accuracy: 95.41899441340782%
Iteration: 3500, Loss: 0.05168115720152855, Accuracy: 95.51353674258702%
Iteration: 4000, Loss: 0.08777405321598053, Accuracy: 95.60807907176623%
Iteration: 4500, Loss: 0.09047064185142517, Accuracy: 95.79716373012462%
Iteration: 5000, Loss: 0.050328612327575684, Accuracy: 95.71121615814353%
Iteration: 5500, Loss: 0.13508306443691254, Accuracy: 95.54791577137946%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1132
Accuracy: 0.9574
Precision: 0.9751
Recall: 0.9465
F1 Score: 0.9606
ROC AUC Score: 0.9897
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 2196647936 bytes
Memory Reserved after model1 deletion: 3093299200 bytes


## **EfficientNet**

In [ ]:
class EfficientNetWithHours(nn.Module):
    def __init__(self, num_hour_features=1):
        super(EfficientNetWithHours, self).__init__()
        # Load a pre-trained EfficientNet-B0 model
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Extract the feature extractor part
        self.features = self.efficientnet.features  # EfficientNet feature extractor

        # The number of features output by the feature extractor
        num_ftrs = self.efficientnet.classifier[1].in_features  # Should be 1280

        # Additional layer for hour feature(s)
        self.hour_fc = nn.Linear(num_hour_features, 16)  # Transform the hour feature

        # Define a new classifier
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(num_ftrs + 16, 1),
        )

    def forward(self, images, hours):
        # Forward pass through EfficientNet feature extractor
        features = self.features(images)
        features = features.mean([2, 3])  # Global average pooling
        # features size: [batch_size, num_ftrs]

        # Forward pass through the hour feature layer
        hour_features = torch.relu(self.hour_fc(hours))  # Output size: [batch_size, 16]

        # Combine the features
        combined_features = torch.cat((features, hour_features), dim=1)  # [batch_size, num_ftrs + 16]

        # Final classification
        output = self.classifier(combined_features)  # Output size: [batch_size, 1]
        return output

learning rate 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 154MB/s]


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.13078011572360992, Accuracy: 92.60850880962613%
Iteration: 1000, Loss: 0.24574053287506104, Accuracy: 93.01246239793726%
Iteration: 1500, Loss: 0.17433631420135498, Accuracy: 93.78599054576708%
Iteration: 2000, Loss: 0.1254563182592392, Accuracy: 93.149978513107%
Iteration: 2500, Loss: 0.1805855929851532, Accuracy: 94.40481306403095%
Iteration: 3000, Loss: 0.10556334257125854, Accuracy: 94.75719810915342%
Iteration: 3500, Loss: 0.1882236897945404, Accuracy: 93.53674258702192%
Iteration: 4000, Loss: 0.12700939178466797, Accuracy: 94.19853889127633%
Iteration: 4500, Loss: 0.21612408757209778, Accuracy: 94.47357112161582%
Iteration: 5000, Loss: 0.11972159147262573, Accuracy: 95.01504082509669%
Iteration: 5500, Loss: 0.07452452182769775, Accuracy: 95.01504082509669%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1390
Accuracy: 0.9479
Precision: 0.9416
Recall: 0.9666
F1 Score: 0.9540
ROC AUC Score: 0.9861
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 117549568 bytes
Memory Reserved after model1 deletion: 195035136 bytes


learning rate 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.1284249722957611, Accuracy: 94.86892995272883%
Iteration: 1000, Loss: 0.18012598156929016, Accuracy: 95.29007305543618%
Iteration: 1500, Loss: 0.09403017908334732, Accuracy: 94.92049849591749%
Iteration: 2000, Loss: 0.046612679958343506, Accuracy: 95.70262140094542%
Iteration: 2500, Loss: 0.11021242290735245, Accuracy: 96.05500644606789%
Iteration: 3000, Loss: 0.03536862134933472, Accuracy: 96.10657498925656%
Iteration: 3500, Loss: 0.08426930010318756, Accuracy: 96.11516974645467%
Iteration: 4000, Loss: 0.15390725433826447, Accuracy: 96.28706489041684%
Iteration: 4500, Loss: 0.10369867086410522, Accuracy: 96.2440911044263%
Iteration: 5000, Loss: 0.19862401485443115, Accuracy: 96.39020197679416%
Iteration: 5500, Loss: 0.08180643618106842, Accuracy: 96.09798023205845%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.0932
Accuracy: 0.9653
Precision: 0.9742
Recall: 0.9634
F1 Score: 0.9687
ROC AUC Score: 0.9917
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 118281728 bytes
Memory Reserved after model1 deletion: 255852544 bytes


learning rate 0.0001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.19425040483474731, Accuracy: 92.72883541039965%
Iteration: 1000, Loss: 0.07972130924463272, Accuracy: 94.01804899011603%
Iteration: 1500, Loss: 0.17188207805156708, Accuracy: 94.80876665234207%
Iteration: 2000, Loss: 0.12325762212276459, Accuracy: 95.0236355822948%
Iteration: 2500, Loss: 0.05338066816329956, Accuracy: 95.22990975504942%
Iteration: 3000, Loss: 0.1347486972808838, Accuracy: 95.40180489901161%
Iteration: 3500, Loss: 0.11200514435768127, Accuracy: 95.82294800171896%
Iteration: 4000, Loss: 0.06026378646492958, Accuracy: 95.88311130210572%
Iteration: 4500, Loss: 0.1474861055612564, Accuracy: 96.063601203266%
Iteration: 5000, Loss: 0.15072090923786163, Accuracy: 95.98624838848302%
Iteration: 5500, Loss: 0.05349541828036308, Accuracy: 95.99484314568113%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1022
Accuracy: 0.9630
Precision: 0.9718
Recall: 0.9617
F1 Score: 0.9667
ROC AUC Score: 0.9912
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 117510144 bytes
Memory Reserved after model1 deletion: 236978176 bytes


learning rate 0.00001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.3217049241065979, Accuracy: 84.40911044263%
Iteration: 1000, Loss: 0.19259116053581238, Accuracy: 89.3940696175333%
Iteration: 1500, Loss: 0.24634677171707153, Accuracy: 90.88096261280619%
Iteration: 2000, Loss: 0.2866280674934387, Accuracy: 91.80060163300386%
Iteration: 2500, Loss: 0.17087391018867493, Accuracy: 92.25612376450366%
Iteration: 3000, Loss: 0.10038313269615173, Accuracy: 92.41082939406962%
Iteration: 3500, Loss: 0.11389091610908508, Accuracy: 93.09840996991835%
Iteration: 4000, Loss: 0.11511431634426117, Accuracy: 93.17576278470133%
Iteration: 4500, Loss: 0.06213150918483734, Accuracy: 93.56252685861624%
Iteration: 5000, Loss: 0.10357673466205597, Accuracy: 93.78599054576708%
Iteration: 5500, Loss: 0.07454939186573029, Accuracy: 93.94929093253116%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1291
Accuracy: 0.9478
Precision: 0.9542
Recall: 0.9523
F1 Score: 0.9532
ROC AUC Score: 0.9862
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 156381184 bytes
Memory Reserved after model1 deletion: 255852544 bytes


## **ConvNext**

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ConvNextWithHours(nn.Module):
    def __init__(self, num_hour_features=1):
        super(ConvNextWithHours, self).__init__()
        # Load a pre-trained ConvNeXt-Small model
        self.convnext = models.convnext_small(pretrained=True)

        # Modify the classifier to exclude the last Linear layer
        self.convnext.classifier = nn.Sequential(
            self.convnext.classifier[0],  # LayerNorm
            self.convnext.classifier[1],  # Flatten
            # Exclude the last Linear layer
        )

        # Obtain num_ftrs from the classifier's LayerNorm layer
        num_ftrs = self.convnext.classifier[0].normalized_shape[0]  # Should be 768

        # Additional layer for hour feature(s)
        self.hour_fc = nn.Sequential(
            nn.Linear(num_hour_features, 16),
            nn.ReLU(inplace=True)
        )

        # Final layer for binary classification
        self.final_fc = nn.Linear(num_ftrs + 16, 1)

    def forward(self, images, hours):
        # Forward pass through ConvNeXt
        convnext_features = self.convnext(images)  # Output shape: [batch_size, num_ftrs]

        # Forward pass through the hour feature layer
        hour_features = self.hour_fc(hours)  # Output shape: [batch_size, 16]

        # Combine the features
        combined_features = torch.cat((convnext_features, hour_features), dim=1)  # Shape: [batch_size, num_ftrs + 16]

        # Final classification
        output = self.final_fc(combined_features)  # Output shape: [batch_size, 1]
        return output


learning rate 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNextWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.3836081624031067, Accuracy: 76.74258702191663%
Iteration: 1000, Loss: 0.36731356382369995, Accuracy: 79.87967339922648%
Iteration: 1500, Loss: 0.3940824270248413, Accuracy: 76.81993983669962%
Iteration: 2000, Loss: 0.38143190741539, Accuracy: 80.05156854318865%
Iteration: 2500, Loss: 0.3344791531562805, Accuracy: 80.32660077352814%
Iteration: 3000, Loss: 0.27782735228538513, Accuracy: 79.87967339922648%
Iteration: 3500, Loss: 0.33815547823905945, Accuracy: 80.0859475719811%
Iteration: 4000, Loss: 0.5515509843826294, Accuracy: 78.85689729265148%
Iteration: 4500, Loss: 0.4260536730289459, Accuracy: 80.12892135797163%
Iteration: 5000, Loss: 0.45773574709892273, Accuracy: 80.12892135797163%
Iteration: 5500, Loss: 0.34237709641456604, Accuracy: 81.22905027932961%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.3922
Accuracy: 0.8035
Precision: 0.8617
Recall: 0.7642
F1 Score: 0.8100
ROC AUC Score: 0.8640
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 851330560 bytes
Memory Reserved after model1 deletion: 2579496960 bytes


learning rate 0.001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNextWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.4274495840072632, Accuracy: 80.72195960464117%
Iteration: 1000, Loss: 0.44885680079460144, Accuracy: 83.85904598195101%
Iteration: 1500, Loss: 0.3224053382873535, Accuracy: 88.28534593897723%
Iteration: 2000, Loss: 0.22669798135757446, Accuracy: 90.38246669531586%
Iteration: 2500, Loss: 0.26580601930618286, Accuracy: 89.77223893425011%
Iteration: 3000, Loss: 0.0857299193739891, Accuracy: 93.52814782982381%
Iteration: 3500, Loss: 0.13706772029399872, Accuracy: 93.08122045552213%
Iteration: 4000, Loss: 0.1726321280002594, Accuracy: 93.82896433175763%
Iteration: 4500, Loss: 0.25839924812316895, Accuracy: 94.78298238074774%
Iteration: 5000, Loss: 0.14148958027362823, Accuracy: 94.49935539321014%
Iteration: 5500, Loss: 0.09929095953702927, Accuracy: 94.25010743446498%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.1362
Accuracy: 0.9430
Precision: 0.9740
Recall: 0.9207
F1 Score: 0.9466
ROC AUC Score: 0.9867
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 852194816 bytes
Memory Reserved after model1 deletion: 2350907392 bytes


learning rate 0.0001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNextWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.148818701505661, Accuracy: 94.49935539321014%
Iteration: 1000, Loss: 0.1373743712902069, Accuracy: 94.74860335195531%
Iteration: 1500, Loss: 0.14345936477184296, Accuracy: 95.83154275891707%
Iteration: 2000, Loss: 0.17154520750045776, Accuracy: 96.11516974645467%
Iteration: 2500, Loss: 0.0897204801440239, Accuracy: 96.08079071766223%
Iteration: 3000, Loss: 0.11211910098791122, Accuracy: 95.59088955737%
Iteration: 3500, Loss: 0.06653853505849838, Accuracy: 96.20971207563386%
Iteration: 4000, Loss: 0.05959409102797508, Accuracy: 96.39020197679416%
Iteration: 4500, Loss: 0.07267892360687256, Accuracy: 95.53932101418135%
Iteration: 5000, Loss: 0.03533121943473816, Accuracy: 96.44177051998281%
Iteration: 5500, Loss: 0.11817114055156708, Accuracy: 95.71121615814353%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.0933
Accuracy: 0.9689
Precision: 0.9646
Recall: 0.9791
F1 Score: 0.9718
ROC AUC Score: 0.9937
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 849681920 bytes
Memory Reserved after model1 deletion: 2134900736 bytes


learning rate 0.00001

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNextWithHours(num_hour_features=1).to(device)
# Calculate pos_weight and initialize BCEWithLogitsLoss with pos_weight
negatives = 34689
positives = 42869
pos_weight = torch.tensor([negatives / positives]).to(device)
error = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Initial setup as before


total_epochs = 7
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
count = 0

# Lists for visualization of loss and accuracy
loss_list = []
iteration_list = []
accuracy_list = []

# Assume optimizer and model are properly initialized
for epoch in range(total_epochs):
    for images, labels, hours in train_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        # Forward pass
        outputs = model(images, hours)
        loss = error(outputs, labels)

        # Zero the gradients
        optimizer.zero_grad()

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        count += 1

        # Testing the model
        if not (count % 50):  # Evaluate every 50 iterations
            total = 0
            correct = 0

            # No gradient tracking needed for evaluation
            with torch.no_grad():
                for images, labels, hours in test_loader:
                    labels = labels.float()
                    images, labels, hours = images.to(device), labels.to(device), hours.to(device)

                    outputs = model(images, hours)
                    predicted = torch.sigmoid(outputs).round()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()  # Ensure labels are correctly shaped

            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if not (count % 500):  # Print every 500 iterations
                print(f"Iteration: {count}, Loss: {loss.item()}, Accuracy: {accuracy}%")

Iteration: 500, Loss: 0.2968873679637909, Accuracy: 93.31327889987108%
Iteration: 1000, Loss: 0.07650311291217804, Accuracy: 95.62526858616243%
Iteration: 1500, Loss: 0.040418121963739395, Accuracy: 95.3072625698324%
Iteration: 2000, Loss: 0.058741066604852676, Accuracy: 95.71121615814353%
Iteration: 2500, Loss: 0.07974477112293243, Accuracy: 96.02922217447357%
Iteration: 3000, Loss: 0.0561225488781929, Accuracy: 96.35582294800172%
Iteration: 3500, Loss: 0.08789858222007751, Accuracy: 96.3300386764074%
Iteration: 4000, Loss: 0.14374113082885742, Accuracy: 96.42458100558659%
Iteration: 4500, Loss: 0.0663643330335617, Accuracy: 96.57928663515256%
Iteration: 5000, Loss: 0.1284179985523224, Accuracy: 96.613665663945%
Iteration: 5500, Loss: 0.03941658139228821, Accuracy: 96.59647614954878%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import numpy as np

model.eval()
val_loss = 0
all_labels = []
all_predictions = []
all_probabilities = []

with torch.no_grad():
    for images, labels, hours in val_loader:
        labels = labels.float()
        images, labels, hours = images.to(device), labels.to(device), hours.to(device)

        outputs = model(images, hours)
        loss = error(outputs, labels)
        val_loss += loss.item()

        probabilities = torch.sigmoid(outputs)
        predicted = probabilities.round()

        all_labels.extend(labels.cpu().numpy().flatten())
        all_predictions.extend(predicted.cpu().numpy().flatten())
        all_probabilities.extend(probabilities.cpu().numpy().flatten())

# Average validation loss
val_loss /= len(val_loader)

# Compute metrics
precision = precision_score(all_labels, all_predictions, zero_division=0)
recall = recall_score(all_labels, all_predictions, zero_division=0)
f1 = f1_score(all_labels, all_predictions, zero_division=0)
accuracy = accuracy_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_probabilities)
cm = confusion_matrix(all_labels, all_predictions)

print(f"\nValidation Set Performance:")
print(f"Validation Loss: {val_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}")
print("Confusion Matrix:")


Validation Set Performance:
Validation Loss: 0.0903
Accuracy: 0.9679
Precision: 0.9669
Recall: 0.9749
F1 Score: 0.9709
ROC AUC Score: 0.9927
Confusion Matrix:


In [ ]:
# Delete model1 and associated variables
del model
del optimizer
#del scheduler  # if you used one
del loss  # if stored

# Clear any cached data
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# (Optional) Verify GPU memory usage
print(f"Memory Allocated after model1 deletion: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved after model1 deletion: {torch.cuda.memory_reserved()} bytes")

Memory Allocated after model1 deletion: 850928128 bytes
Memory Reserved after model1 deletion: 2327838720 bytes


## **Plot**

In [ ]:
import pandas as pd

# Your data
model_name = ['VGG-16', 'ResNet18', 'ResNet50', 'EfficientNet', 'ConvNeXt']
learning_rates = [0.01, 0.001, 0.0001, 0.00001]

accuracy_1 = [80.30, 94.51, 94.98, 95.74]       # For VGG-16
accuracy_2 = [93.94, 95.62, 94.27, 95.62]       # For ResNet18
accuracy_3 = [94.21, 93.66, 96.31, 95.80]       # For ResNet50
accuracy_4 = [94.79, 96.53, 96.30, 94.78]       # For EfficientNet
accuracy_5 = [80.35, 94.30, 96.89, 96.79]       # For ConvNeXt

# Create a DataFrame for accuracy
accuracies = [accuracy_1, accuracy_2, accuracy_3, accuracy_4, accuracy_5]
df_accuracy = pd.DataFrame(accuracies, index=model_name, columns=learning_rates)

# Similarly for F1 scores
F1_1 = [0.8084, 0.9508, 0.9528, 0.9606]
F1_2 = [0.9471, 0.9602, 0.9464, 0.9609]
F1_3 = [0.9480, 0.9463, 0.9665, 0.9617]
F1_4 = [0.9540, 0.9687, 0.9667, 0.9532]
F1_5 = [0.8100, 0.9466, 0.9718, 0.9709]

# Convert F1 scores to percentages for consistency
F1_scores = [[score * 100 for score in lst] for lst in [F1_1, F1_2, F1_3, F1_4, F1_5]]
df_F1 = pd.DataFrame(F1_scores, index=model_name, columns=learning_rates)
df_accuracy.columns = ['1e-2', '1e-3', '1e-4', '1e-5']
df_F1.columns = ['1e-2', '1e-3', '1e-4', '1e-5']
sns.set_context("talk", font_scale=1.2)
sns.set_style("whitegrid")

# Plot Accuracy Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_accuracy, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5, linecolor='gray')
plt.xlabel('Learning Rate', fontsize=18)
plt.title('Model Accuracy Heatmap (%)', fontsize=18)
plt.yticks(rotation=0, fontsize=16)
plt.xticks(rotation=45, fontsize=16)
plt.tight_layout()
plt.show()

# Plot F1 Score Heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_F1, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5, linecolor='gray')
plt.xlabel('Learning Rate', fontsize=18)
plt.title('Model F1 Score Heatmap (%)', fontsize=18)
plt.yticks(rotation=0, fontsize=16)
plt.xticks(rotation=45, fontsize=16)
plt.tight_layout()
plt.show()